# MLQC Updates & Progress

---


> #### _TODO's:_
> 
> 1. Add NN Architecture Diagram to Method Overview section
> 2. General improvements to method overview: conciseness, math, etc
> 3. NN Training & Performance Plots
> 4. Hindcasting plots
> 5. DNS data plots


## Method Overview

**Baseline**: There will be noise in the data. Most of the time it comes from the observation environment. If we want to know if the data are useful or erroneous, we need to characterize the contribution of the environmental noise and distinguish this from defects like bad readings and sensor failures in a statistically meaningful way.

**Steps:**

1. _Data:_ Viewed as an entire collection to better utilize relationships.

    - Equations of State are used to model how variables are related
    - Doesn't require full lat/lon grid, which allows data to be viewed locally
    - Handles higher resolutions well since obs don't need to be indexed by lower-res grids
>
2. _Equations of State (EoS):_ Stochastic Partial Differential Equations (SPDE) + Numerical Solver

    - commonly used for both spatial statistics and modeling uncertainty in dynamic relationships
    - natural framework for uncertainty quantification (UQ), probability, and correlation analyses
    - How much fluctuation is in the signal? How much of that has to do with what we expect things to look like?
>
3. _NN:_ quickly perform analyses by replicating EoS relationships 

    - Model A: Neural SPDE, Operators, etc can replicate equation solutions
    - Model B: Expected noise evolution can check the model is fitting the correct system
    - Model A+B (GAN): train a generative model that compares proposed solutions and their statistics to known statistics
>
4. _Application:_ QC

    - given a dataset, quantify cross-correlations + uncertainty between variables
        - other quantities: continuity, rate-of-change, large deviation ("spikiness")
    - given a threshold/tolerance, rank values by analyzed quantity metrics

### NN

#### Steps to Formulating a NN to "Solve" the SPDE System

1. Specify system including GAN metric
>
2. Direct Numerical Solve (DNS) of system for initial training
    - _Need to know NN actually works_
>
3. Train one sub-net per equation on DNS data
>
4. Integrate sub-nets into larger model by training FNN layer on simultaneous outputs
>
5. Test on SD data, tune as needed
    - _this needs to be done carefully to avoid bias (see below)_

#### Avoiding Bias

When performing these kinds of statistical analyses, it's useful to distinguish the difference between bias and correctly estimating the joint distribution over the quantities of interest. A "correct" model will adapt to changing values/statistics irrespective of input data. This could still mean that it underperforms on certain behaviors like larger-than-expected fluctuations (e.g. caused by weather events). A biased model is accurate on only a subset of data. 

>***Example:***
>
> Suppose the NN generally performs better on TPOS mission data than hurricane monitoring data. Then, we would say it is biased towards the TPOS values (conv. away from hurricane values). On the other hand, if the model performs similarly between the two with the exception that it is unreliable for measurements taken during a hurricane, then we have a model that doesn't handle extreme weather very well but otherwise manages to generalize to other regions of the ocean. This is an example of the model performance being more fragile than we'd like, which can be addressed in a relatively straightforward way (_see below_).

**DOs**: 

- check related quantities are correlated 
- verify noise distribution is similar to DNS training data
- no overfitting between points (ie - out of sample comparisons)
>

**DONTs**: 

- tune using only SD data that didn't perform well
- limit model validation by performance on individual datasets or specific subsets of interest
- **VALIDATE ON TRAINING DATA!!!**

#### Generalizability & Accounting for Unseen Observation Conditions

The poor performance in the example above can be accounted for by including hurricane-like solutions to the EoS in the DNS data creation and then incorporating those results into the train/test datasets. This allows for further validation against conditions where poor-performance was seen without simply re-tuning the model on the out-of-sample data. In-turn, this means the robustness (generalizability) of the model can be assessed for similar cases in future usage.
>
On top of improved compute time, implementing the EoS in a NN also means that we can easily deploy it across datasets from other oceanographic regions and sensor platforms.

<!-- TODO: ![NN architecture diagram](nnarch.png) -->

### EoS System and USV Sampling

To begin testing this method, we choose the Gibbs Seawater Model as our System of Equations. This provides an easy starting point since this model is widely used in the Oceanographic Community to represent the non-equilibrium thermodynamic relationships between Salinity, Temperature, Pressure, Current Velocity, and Seawater Density.
>
Rather than directly modeling the turbulent dynamics, we think of these quantities as random fields that are primarily influenced by a randomly driven velocity field. This results in a system of SPDE's that co-evolve in space and time, and so each value we observe can be thought of as belonging to a distribution at that time and location. This allows us to evaluate the data we collect at the resolution native to it, rather than relying on exogenous datasets that may not be directly comparable. Furthermore, we can now compute the joint distribution of several observation variables. Together, these allow us to view fluctuations in the data in terms of their physical likelihood, which means that a successful model should be agnostic to variations encountered while a USV is underway.
>
Importantly, we simulate the system at a higher resolution than the sampling data. In the case of USV's, the ocean is evolving underneath the platform while it's in motion. A higher resolution means that we have sufficient resolution to represent the distribution of a variable sampled at that location.

### DNS

To ensure the NN learns to solve the system of equations we specify, we begin by training it on DNS data of that system. We also formulate a physics-based loss function that penalizes the NN predictions by comparing against the equation governing the evolution of the multi-dimenstional probability distribution that can be directly derived from this system. This allows us to always interpret the NN predictions in terms of physics, but it also means that the NN absolutely needs to function as intended.
>
The DNS solver is a straightforward spectral Galerkin solver. To avoid bias, we initially generate our numerical training data randomly by sampling it from an irregular, spatially correlated (i.e. anisotropic) distribution over the solver grid. The training set is made by slicing this grid into subsets, which prevents the model learning the boundary conditions of the solver - which don't appear in real data.
>
If we have captured the system correctly, we should expect to see similar features and behaviors to real data such as time evolution of the statistical moments.

![Numerical Data Generation](../pics/fieldsim.png)

## Timeline

1. **Sub-Model Development & Training** 
- STATUS: $\checkmark$
>
2. **Whole model Development & Training** 
- STATUS: $\checkmark$
>
3. **Model evaluation** (*theoretical consistency*) 
- STATUS: $\checkmark$
>
4. **Model evaluation** (*application:* Saildrone QC) 
- STATUS: $\checkmark$
>
5. **Model evaluation** (*application:* Data hindcast) 
- STATUS: **in progress**
>
6. **Test case** (*application:* IOOS glider ERDDAP datasets) 
- STATUS: **upcoming**
>
7. **Test case** (*application:* profiler data - Paige Lavin) 
- STATUS: **upcoming**
>
8. **Test case** (*application:* remote sensing data - Paige Lavin) 
- STATUS: **upcoming**
>
9. **Project evaluation** 
- STATUS: TBD
>

### Known Saildrone Data Issues

| ISSUE                         | Mission year   | Drone | time period                                                     |  
|-------------------------------|:--------------:|-------|-----------------------------------------------------------------|
| Spuriously low salinity       | Arctic 2019    |  1037 | Intermittent May 15-23, especially 23rd                         |
| jump/drift in salinity and O2 | Arctic 2019    |  1041 | ~5/29 0500 UTC                                                  |
| salinity drift                | Arctic 2019    |  1037 | starting 6/7 0300 UTC                                           |
| funky salinity/O2             | Arctic 2019    |  1035 | starting around 7/12 2345 UTC                                   |
| salinity                      | Arctic 2019    |  1034 | ~7/13 0000 UTC                                                  |
| RH pegged near 100%           | Arctic 2019    |  1037 | from Jun '19 through rest of mission                            |
| wind dir & speed              | Hurricane 2022 |  1032 | 8/4-8/13 had wind sensor issues that caused vehicle recall      |
| AT/RH & SBE37 bad             | Hurricane 2022 |  1078 | clearly spurious measurements after sensors turned back on 8/13 |

## Progress

We apply the QARTOD flagging schema to unseen Saildrone data and assign those flags to observations based on the spatially & temporally evolving joint distribution generated by the model for the set observations and variables. 
>
In the case below, the NN computes the joint distribution between salinity and temperature in the presence of noisy fluid motion coupled to an environment.
>
The data are then flagged based on a set of tolerance parameters representing degrees of deviation from the expected range, or, where in the predicted distribution the data lie.

#### QARTOD Primary Flags

| FLAG       | Value |
|------------|:-----:|
| GOOD       |   1   |
| UNKNOWN    |   2   |
| SUSPECT    |   3   |
| FAIL       |   4   |
| MISSING    |   9   |

### Testing Flag Results

In [ ]:
# !pip install pandas plotly

In [1]:
import pandas as pd
import plotly.express as px

import sys
mlqcdir = '..'

In [2]:
hi1090qc = pd.read_csv(f'{mlqcdir}/data/sd1090_hi23_STqc_dist_test_flags.csv')

As a sanity check, let's see what happens to the flags in SST as we vary the tolerance parameters...

| Test       | Parameters      | Description                             |
|------------|:---------------:|-----------------------------------------|
| Test 1     | [1.0, 2.0, 3.0] | Regular intervals between flags         |
| Test 2     | [2.0, 4.0, 6.0] | Larger, regular intervals between flags |
| Test 3     | [1.5, 2.5, 4.0] | Skewed intervals between flags          |
| Test 4     | [1.0, 2.0, 3.0] | Test 1 on normalized data               |
| Test 5     | [2.0, 4.0, 6.0] | Test 2 on normalized data               |
| Test 6     | [1.5, 2.5, 4.0] | Test 3 on normalized data               |


#### Test 1 - Temperature

In [26]:
px.scatter(
    hi1090qc, 
    x="time", y="TEMP_SBE37_MEAN", 
    color='SST QC Flag: Test 1', 
    width = 1400, height = 800, 
    color_discrete_sequence=["green", "gray", "goldenrod", "red", "black"], 
    category_orders={"SST QC Flag: Test 1": ["GOOD", "UNKNKOWN", "SUSPECT", "FAIL", "MISSING"]},
    title='SD-1090 HI Chem 23 SST'
    )

#### Test 2 - Temperature

In [29]:
px.scatter(
    hi1090qc, 
    x="time", y="TEMP_SBE37_MEAN", 
    color='SST QC Flag: Test 2', 
    width = 1400, height = 800, 
    color_discrete_sequence=["green", "gray", "goldenrod", "red", "black"], 
    category_orders={"SST QC Flag: Test 2": ["GOOD", "UNKNKOWN", "SUSPECT", "FAIL", "MISSING"]},
    title='SD-1090 HI Chem 23 SST'
    )

#### Test 3 - Temperature

In [30]:
px.scatter(
    hi1090qc, 
    x="time", y="TEMP_SBE37_MEAN", 
    color='SST QC Flag: Test 3', 
    width = 1400, height = 800, 
    color_discrete_sequence=["green", "gray", "goldenrod", "red", "black"], 
    category_orders={"SST QC Flag: Test 3": ["GOOD", "UNKNKOWN", "SUSPECT", "FAIL", "MISSING"]},
    title='SD-1090 HI Chem 23 SST'
    )

#### Test 3 - Salinity: Check for Flag Similarity

Assuming we're happy with the Test 3 parameters, do we see a correspondence between correlated variables (as we should)?
>
> __Note:__ Double clicking the flag in the key to the right of the plot will isolate those points, making it easier to compare the plots.

In [38]:
# Any hints as to why I'm getting the wrong color on the UKNOWN point?
px.scatter(
    hi1090qc, 
    x="time", y="SAL_SBE37_MEAN", 
    color='SSS QC Flag: Test 3', 
    width = 1400, height = 800, 
    color_discrete_sequence=["green", "gray", "goldenrod", "red", "black"], 
    category_orders={"SSS QC Flag: Test 3": ["GOOD", "UNKNKOWN", "SUSPECT", "FAIL", "MISSING"]},
    title='SD-1090 HI Chem 23 SSS'
    )

Clearly it's not a perfect correspondence, but there is also some obvious consistency between the two.

#### Test 6 - Salinity

Finally, we can compare the flags assigned (using identical tolerance parameters) to the normalized data, which contains the actual values the NN used to generate the distribution. If the assigned flags are not identical, it's likely time to go back to the drawing board.
>
Since normalization is a linear transformation $x^* = (x - \mu) / \sigma$, it should preserve the shape of the distribution. If what the NN generates doesn't behave this way, then whatever the output is can't be said to be a distribution, and may not even represent EoS we think it does - in other words: the model isn't trustworthy. 
>
It's worth noting, however, that any classification scheme applied to continuous values can and will result in values near the boundary of the respective classes. One or two minor discrepancies may not mean much if they are close to the threshold. This is true if the parameters are chosen arbitrarily (as in Tests 1-6), or selected more carefully (e.g. by using parameter optimization).

In [39]:
px.scatter(
    hi1090qc, 
    x="time", y="SAL_SBE37_MEAN", 
    color='SSS QC Flag: Test 6', 
    width = 1400, height = 800, 
    color_discrete_sequence=["green", "gray", "goldenrod", "red", "black"], 
    category_orders={"SSS QC Flag: Test 6": ["GOOD", "UNKNKOWN", "SUSPECT", "FAIL", "MISSING"]},
    title='SD-1090 HI Chem 23 SSS (normalized)'
    )